## Consigna

Columna “reservation_status_date” (CARLA)

- Limpieza y transformación: 
    -  Esta columna es de tipo object y deberíamos pasarla a tipo date. Para poder hacerlo, vamos a necesitar unificar el formato en el que aparecen las fechas (en algunos casos tienen guiones y en otros barras).
    - Hay 30 filas con fechas posteriores a 2017. En principio no nos preocupa mucho por tratarse de un porcentaje muy pequeño. Intentaremos corregirlo teniendo en cuenta las columnas que tienen información de la fecha de arrival. 

- Imputación: 
    - Completar a partir de “arrival _date_year”
    - El resto imputarlas con KNN o iterative.


## Exploración y análisis

In [1]:
# importar librerías 

import pandas as pd
import numpy as np

In [2]:
# guardar datos en un dataframe

df = pd.read_csv("data/finanzas_hotel_bookings.csv", index_col=0)

/tmp/ipykernel_6155/1059111946.py:1: DtypeWarning: Columns (1,2,5,13,14,15,16,20,21,26,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/finanzas_hotel_bookings.csv", index_col=0)


In [37]:
# definir una función para explorar las caraterísticas de una columna rápidamente

def quick_check(df, column_name):
        """ This function is for testing purposes, to quicky check data type and unique values of a column"""
        print (f"\nColumn name: {column_name}")
        print (f"\nData type: {df[column_name].dtype}")
        print (f"\nNot null count: {df[column_name].notnull().sum()}")
        print (f"\nNull count: {df[column_name].isnull().sum()}, {round(df[column_name].isnull().sum()/df.shape[0]*100)}%")
        print (f"\nDuplicated values: {df[column_name].duplicated().sum()}")
        print (f"\nUnique values: \n{df[column_name].unique()}")
        print (f"\nValue counts: \n{df[column_name].value_counts()}")
        
        

In [15]:
# revisar la columna "reservation_status_date"

quick_check(df,"reservation_status_date")


Column name: reservation_status_date

Data type: object

Not null count: 106514

Null count: 76363, 42%

Duplicated values: 181885

Unique values: 
['2015-07-01 00:00:00' '2015-07-02 00:00:00' '2015-07-03 00:00:00'
 '2015-05-06 00:00:00' '2015-04-22 00:00:00' '2015-06-23 00:00:00'
 '2015-07-05 00:00:00' nan '2015-07-07 00:00:00' '2015-07-08 00:00:00'
 '2015-05-11 00:00:00' '2015-07-15 00:00:00' '2015-07-16 00:00:00'
 '2015-05-29 00:00:00' '2015-05-19 00:00:00' '2015-06-19 00:00:00'
 '2015-07-06 00:00:00' '2015-05-23 00:00:00' '2015-05-18 00:00:00'
 '2015-07-09 00:00:00' '2015-06-02 00:00:00' '2015-07-04 00:00:00'
 '2015-06-29 00:00:00' '2015-06-16 00:00:00' '2015-06-18 00:00:00'
 '2015-06-12 00:00:00' '2015-06-09 00:00:00' '2015-05-26 00:00:00'
 '2015-07-11 00:00:00' '2015-07-12 00:00:00' '2015-07-13 00:00:00'
 '2015-07-17 00:00:00' '2015-04-15 00:00:00' '2015-05-13 00:00:00'
 '2015-07-10 00:00:00' '2015-05-20 00:00:00' '2015-05-12 00:00:00'
 '2015-07-14 00:00:00' '2015-06-17 00:00:00

In [17]:
# Conclusiones parciales: 
    # Algunas fechas tienen formato fecha y hora '2015-06-10 00:00:00' y otras solo fecha '2023-04-31'
    # En el caso de las que tienen fecha y hora, la hora es siempre '00:00:00'
# filtrar filas que tienen el formato '2023-04-31' 

df_filtered = df[df['reservation_status_date'].str.match(r'^\d{4}-\d{2}-\d{2}$', na=False)]

In [18]:
# revisar columna 'reservation_status_date' del df filtrado
quick_check(df_filtered, 'reservation_status_date') 


Column name: reservation_status_date

Data type: object

Not null count: 45

Null count: 0, 0%

Duplicated values: 0

Unique values: 
['2030-02-30' '2030-11-31' '2029-04-31' '2030-06-31' '2030-09-31'
 '2025-02-30' '2029-11-31' '2023-06-31' '2029-06-31' '2029-02-30'
 '2022-09-31' '2024-09-31' '2028-04-31' '2024-06-31' '2025-11-31'
 '2026-09-31' '2027-09-31' '2023-11-31' '2028-09-31' '2028-02-30'
 '2026-06-31' '2024-04-31' '2023-04-31' '2025-04-31' '2027-04-31'
 '2022-06-31' '2024-02-30' '2025-09-31' '2027-02-30' '2030-04-31'
 '2023-09-31' '2027-06-31' '2026-11-31' '2028-11-31' '2028-06-31'
 '2026-04-31' '2022-11-31' '2025-06-31' '2029-09-31' '2023-02-30'
 '2026-02-30' '2022-04-31' '2022-02-30' '2027-11-31' '2024-11-31']

Value counts: 
2030-02-30    1
2025-04-31    1
2022-06-31    1
2024-02-30    1
2025-09-31    1
2027-02-30    1
2030-04-31    1
2023-09-31    1
2027-06-31    1
2026-11-31    1
2028-11-31    1
2028-06-31    1
2026-04-31    1
2022-11-31    1
2025-06-31    1
2029-09-31    

In [19]:
# inspeccionar df filtrado 

df_filtered

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
313,Resort Hotel,False,90.0,NaN,July,29.0,12.0,2.0,5.0,2.0,...,5.0,NaN,0.0,NaN,117.22,0.0,1.0,Check-Out,2030-02-30,NaN
7646,Resort Hotel,True,165.0,NaN,August,34.0,18.0,0.0,3.0,2.0,...,240.0,NaN,0.0,Transient,222.00,0.0,0.0,Canceled,2030-11-31,NaN
9139,Resort Hotel,True,16.0,NaN,October,NaN,29.0,0.0,1.0,2.0,...,NaN,NaN,0.0,Transient,38.00,0.0,1.0,Canceled,2029-04-31,NaN
9493,Resort Hotel,True,183.0,NaN,December,NaN,3.0,2.0,1.0,2.0,...,38.0,NaN,0.0,NaN,72.00,0.0,0.0,Canceled,2030-06-31,NaN
18350,Resort Hotel,True,82.0,2016.0,2,7.0,7.0,2.0,0.0,2.0,...,67.0,NaN,0.0,NaN,86.00,0.0,0.0,Canceled,2030-09-31,NaN
21580,Resort Hotel,False,94.0,2016.0,March,10.0,4.0,2.0,2.0,51.0,...,6.0,NaN,0.0,Transient,4.00,0.0,2.0,Check-Out,2025-02-30,NaN
22854,Resort Hotel,False,7.0,2016.0,April,15.0,5.0,0.0,4.0,1.0,...,NaN,53.0,0.0,Transient,42.00,0.0,0.0,Check-Out,2029-11-31,NaN
24063,Resort Hotel,False,46.0,NaN,May,20.0,8.0,2.0,2.0,2.0,...,95.0,NaN,0.0,Transient,54.20,0.0,1.0,Check-Out,2023-06-31,NaN
24395,Resort Hotel,False,1.0,2016.0,May,21.0,19.0,0.0,1.0,1.0,...,NaN,NaN,0.0,Transient,42.00,0.0,0.0,Check-Out,2029-06-31,NaN
34886,Resort Hotel,False,25.0,2017.0,April,13.0,1.0,2.0,5.0,2.0,...,240.0,NaN,0.0,NaN,155.00,1.0,0.0,Check-Out,2029-02-30,NaN


In [20]:
# conocer la cantidad de filas que tiene el df filtrado

df_filtered.shape

(45, 32)

## Conclusiones 

- Todas las fechas del df filtrado (45) son erróneas ya que contienen días que no existen en el mes, como 31/09 o 30/02
- Además, hay varias que tienen años en el futuro
- Decisión: reemplazar las fechas por una categoría diferente parfa que no intervengan negativamente en el análisis


## Acciones de limpieza y transformación

1. Reemplazar los nulos con "0000-00-00"
2. Reemplazar las fechas erróneas con "0000-00-00"
3. Cambiar el formato de la columna a "datetime" (usar to datetime)


In [34]:
# 1.Reemplazar los nulos con "0000-00-00"

def imputar_fechas_nulas(df, columns):
        """ Dada una lista de columnas y un dataframe, esta función completa los nulos en las columnas con "0000-00-00".
        """
        # Iterar sobre la lista de columnas y reemplazar nulos por "0000-00-00"
        for column in columns:
            # verifica si la columna existe en el dataframe
            if column in df.columns:
                # Reemplaza los nulos con "0000-00-00"
                df[column] = df[column].fillna("0000-00-00")
            # si la columna no existe imprime un mensaje
            else:
                print(f"❌ La columna '{column}' no existe en el dataframe dado")
        return df


In [35]:
columns = ['reservation_status_date']
imputar_fechas_nulas(df, columns)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
0,Resort Hotel,False,342.000000,2015.0,July,27.0,1.0,0.000000,0.000000,2.0,...,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
1,Resort Hotel,False,737.000000,NaN,July,27.0,1.0,0.000000,0.000000,2.0,...,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
2,Resort Hotel,False,7.000000,2015.0,July,27.0,1.0,0.000000,1.000000,1.0,...,NaN,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
3,Resort Hotel,False,13.000000,NaN,July,27.0,1.0,0.000000,1.000000,1.0,...,304.0,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
4,Resort Hotel,False,14.000000,NaN,July,NaN,1.0,0.000000,2.000000,2.0,...,240.0,NaN,0.0,Transient,98.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182872,NaN,NaN,224.337762,NaN,NaN,NaN,NaN,19.005545,19.819823,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0000-00-00,NaN
182873,NaN,NaN,390.141963,NaN,NaN,NaN,NaN,14.751794,19.989726,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0000-00-00,NaN
182874,NaN,NaN,230.689826,NaN,NaN,NaN,NaN,11.409496,20.461372,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0000-00-00,NaN
182875,NaN,NaN,304.888534,NaN,NaN,NaN,NaN,16.744472,15.400773,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0000-00-00,NaN


In [38]:
quick_check(df, 'reservation_status_date')


Column name: reservation_status_date

Data type: object

Not null count: 182877

Null count: 0, 0%

Duplicated values: 181885

Unique values: 
['2015-07-01 00:00:00' '2015-07-02 00:00:00' '2015-07-03 00:00:00'
 '2015-05-06 00:00:00' '2015-04-22 00:00:00' '2015-06-23 00:00:00'
 '2015-07-05 00:00:00' '0000-00-00' '2015-07-07 00:00:00'
 '2015-07-08 00:00:00' '2015-05-11 00:00:00' '2015-07-15 00:00:00'
 '2015-07-16 00:00:00' '2015-05-29 00:00:00' '2015-05-19 00:00:00'
 '2015-06-19 00:00:00' '2015-07-06 00:00:00' '2015-05-23 00:00:00'
 '2015-05-18 00:00:00' '2015-07-09 00:00:00' '2015-06-02 00:00:00'
 '2015-07-04 00:00:00' '2015-06-29 00:00:00' '2015-06-16 00:00:00'
 '2015-06-18 00:00:00' '2015-06-12 00:00:00' '2015-06-09 00:00:00'
 '2015-05-26 00:00:00' '2015-07-11 00:00:00' '2015-07-12 00:00:00'
 '2015-07-13 00:00:00' '2015-07-17 00:00:00' '2015-04-15 00:00:00'
 '2015-05-13 00:00:00' '2015-07-10 00:00:00' '2015-05-20 00:00:00'
 '2015-05-12 00:00:00' '2015-07-14 00:00:00' '2015-06-17 00:0

In [ ]:
# 2. Reemplazar las fechas erróneas con "0000-00-00"